In [43]:
from random import randint, randrange
import numpy as np
import math
from math import fabs
from numba import jit, njit,prange, vectorize, cuda, guvectorize,int32,float32, uint16

In [44]:
#!find / -iname 'libdevice'
#!find / -iname 'libnvvm.so'

In [45]:
import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/local/cuda-10.1/nvvm/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/local/cuda-10.1/nvvm/lib64/libnvvm.so"

In [46]:
angles = 18
nofl = 1
eventsperfl = 100
dig16codes = 70
elms = 2
TPB = 16
blocks_per_grid = 64

In [47]:
ar162stress = np.random.rand(dig16codes,)
data = np.random.rand(elms,dig16codes).astype(np.float32)

In [48]:
def createfl():
    fl_code16 = []
    numberevents = 0
    maxlength = 0
    for fl in range(nofl):
        events = randint(eventsperfl-3,eventsperfl+3)
        if events > maxlength : maxlength = events
        numberevents += events
        arfl = np.empty(shape=(events,),dtype=np.int32)
        for event in range(events):
            code16 = randrange(dig16codes)
            arfl[event] = code16
        fl_code16.append(arfl)
    return fl_code16, maxlength
fl_code16, maxlength = createfl()

In [49]:
def getfldefar(nofl,maxlength,fl_code16):
    fldefar = np.full(shape=(nofl,maxlength),fill_value=-1,dtype=np.int32)
    for i,ar in enumerate(fl_code16):
        events = ar.shape[0]
        fldefar[i,:events] = ar[:]
    return fldefar
fldefar = getfldefar(nofl,maxlength,fl_code16)
fldefar.shape

(1, 97)

In [50]:
@cuda.jit(device=True)
def reversals(arfl,sth):
    fl_len = arfl.shape[0]
    j = 0
    x, x_last, d_next, d_last = 0.0, 0.0, 0.0, 0.0
    for i in range(fl_len):
        code16 = arfl[i]
        if code16 == -1:
          break
        x_next = sth[i]
        if i == 0:
          x_last = x_next
          continue
        if i == 1:
          x = x_next
          d_last = x - x_last
          sth[0] = x_last
          j += 1
        d_next = x_next - x
        if d_last * d_next < 0.0:
          sth[j] = x
          j += 1
        x_last = x
        x = x_next
        d_last = d_next
    sth[j] = x
    j += 1
    return sth, j

In [51]:
@cuda.jit(device=True)
def find_rainflow_cycles(series):
    revs = reversals(series)
    result1 = [np.float32(x) for x in range(0)]
    result2 = [np.float32(x) for x in range(0)]
    residue = [np.float32(x) for x in range(0)]
    len_residue = 0
    for reversal in revs:
        residue.append(reversal)
        len_residue += 1
        while len_residue >= 4:
            S0, S1, S2, S3 = residue[-4], residue[-3], residue[-2], residue[-1]
            dS1, dS2, dS3 = fabs(S1-S0), fabs(S2-S1), fabs(S3-S2)
            if (dS2 <= dS1) and (dS2 <= dS3):
                result1.append(S1)
                result2.append(S2)
                last = residue.pop()
                residue.pop()
                residue.pop()
                residue.append(last)
                len_residue -= 2
            else:
                break
    return result1, result2

In [52]:
@cuda.jit(device=True)
def getstress(ar162stress,code16):
    stress = ar162stress[code16]
    return stress

In [53]:
@cuda.jit(device=True)
def iterflights(angles,ar162stress,arfls,sa_in,tx):
    flights = arfls.shape[0]
    #sth = cuda.local.array(shape=2000,dtype=float32)
    dam = 0.0
    for angle in range(angles):
        for fl in range(flights):
            arfl = arfls[fl]
            fl_len = arfl.shape[0]
            #start reversals
            j = 0
            #sa_ind = 0
            code16 = 0
            #lenps = 1
            x, x_last, d_next, d_last = 0.0, 0.0, 0.0, 0.0
            for i in range(fl_len):
                code16next = arfl[i]
                if code16next == -1:
                    break
                x_next = getstress(ar162stress,code16next)
                if i == 0:
                    x_last = x_next
                    code16last = code16next
                    continue
                if i == 1:
                    x = x_next
                    code16 = code16next
                    d_last = x - x_last
                    sa_in[tx,0] = code16last # j= 0
                    #sth[0] = x_last
                    j += 1
                    continue
                d_next = x_next - x
                d_mul = d_last * d_next
                if (d_mul < 0.0) or (i == fl_len-1):
                    # start rainflow
                    #sth[j] = x
                    if d_mul < 0.0:
                        sa_in[tx,j] = code16
                    if i == fl_len-1:
                        if d_mul < 0.0:
                            j += 1
                        sa_in[tx,j] = code16next
                    #lenps += 1
                    while j > 3:
                        ix0,ix1 = sa_in[tx,j-3],sa_in[tx,j-2]
                        ix2 = sa_in[tx,j-1]
                        s0 = getstress(ar162stress,ix0)
                        s1 = getstress(ar162stress,ix1)
                        s2 = getstress(ar162stress,ix2)
                        s3 = x
                        dS1, dS2, dS3 = fabs(s1-s0), fabs(s2-s1), fabs(s3-s2)
                        if (dS2 <= dS1) and (dS2 <= dS3):
                            dam += s1 + s2
                            sa_in[tx,j-3] = code16
                            j -= 2
                            #lenps -= 2
                        else:
                            break
                    # end rainflow
                    j += 1
                x_last = x
                x = x_next
                d_last = d_next
                code16 = code16next
            #sth[j] = x
            #j += 1
            k = 0
            while j > 4:
                ix0, ix1 = sa_in[tx,k], sa_in[tx,k+1]
                s0 = getstress(ar162stress,ix0)
                s1 = getstress(ar162stress,ix1)
                dam += s0 + s1
                j -= 2
                k += 2
            #end reversals
            # end flight
    return dam

In [64]:
@cuda.jit
def genseq1dsa(arst,arfls,angles,out):
    noelms = arst.shape[0]
    #maxev = arfls.shape[1]
    #TPB = cuda.blockDim
    rf_index = cuda.shared.array(shape=(16, 1000), dtype=uint16)
    pos = cuda.grid(1)
    stride = cuda.gridsize(1)
    tx = cuda.threadIdx.x
    if pos < noelms:
        ar162stress = arst[pos]
        dam = iterflights(angles,ar162stress,arfls,rf_index,tx)
        #sth_sa[tx,0] = 1.
        out[pos] = dam
        #cuda.syncthreads()

In [65]:
arst_device = cuda.to_device(data)
fldef_device = cuda.to_device(fldefar)
arst_device.shape

(2, 70)

In [66]:
out_device = cuda.device_array(shape=(elms,), dtype=np.float32)
out_device.shape

(2)

In [67]:
%%time
threads_per_block = 16
blocks_per_grid = 64
genseq1dsa[blocks_per_grid, threads_per_block](arst_device,fldef_device,angles,out_device)

out = out_device.copy_to_host()

CPU times: user 1.85 s, sys: 64.8 ms, total: 1.91 s
Wall time: 1.89 s


In [68]:
out

array([579.96234, 447.5969 ], dtype=float32)

In [69]:
for i in range(0,10,2):
    print(i)

0
2
4
6
8


In [70]:
np.zeros(shape=(32, 500), dtype=np.float32).nbytes

64000

## test float shared array

In [71]:
@cuda.jit(device=True)
def iterflights2(angles,ar162stress,arfls,sa_in,tx):
    flights = arfls.shape[0]
    #sth = cuda.local.array(shape=2000,dtype=float32)
    dam = 0.0
    for angle in range(angles):
        for fl in range(flights):
            arfl = arfls[fl]
            fl_len = arfl.shape[0]
            #start reversals
            j = 0
            #sa_ind = 0
            code16 = 0
            #lenps = 1
            x, x_last, d_next, d_last = 0.0, 0.0, 0.0, 0.0
            for i in range(fl_len):
                x_next = arfl[i]
                dam += x_next
    return dam

In [83]:
@cuda.jit
def genseq1dsafloat(arst,arfls,angles,out):
    height = arst.shape[0]
    maxev = arfls.shape[1]
    #TPB = cuda.blockDim
    sa_float = cuda.shared.array(shape=(TPB, 400), dtype=float32)
    pos = cuda.grid(1)
    tx = cuda.threadIdx.x
    if pos < height:
        sa_float[tx,0:dig16codes] = arst[pos,:]
        print(arst[pos,:])
        dam = iterflights2(angles,ar162stress,arfls,sa_float,tx)
        #sth_sa[tx,0] = 1.
        out[pos] = dam
        #cuda.syncthreads()

In [84]:
genseq1dsafloat[blocks_per_grid, TPB](arst_device,fldef_device,angles,out_device)

out = out_device.copy_to_host()

In [80]:
data[0]

array([0.30695653, 0.93903923, 0.7480402 , 0.2463979 , 0.54415905,
       0.56764317, 0.6969538 , 0.38104752, 0.62823594, 0.41107342,
       0.2230039 , 0.98501354, 0.14190213, 0.14087997, 0.9267583 ,
       0.03186223, 0.7166828 , 0.46855736, 0.7798468 , 0.7792983 ,
       0.49968502, 0.04472446, 0.10498572, 0.8900595 , 0.02351243,
       0.74466616, 0.48601314, 0.34843686, 0.8180687 , 0.31792614,
       0.6319939 , 0.801381  , 0.5703511 , 0.81168556, 0.5426711 ,
       0.22536607, 0.30370837, 0.17861907, 0.2527725 , 0.11529436,
       0.902883  , 0.53485674, 0.09669375, 0.8205226 , 0.24087733,
       0.8466535 , 0.9553601 , 0.01155773, 0.4592039 , 0.96196616,
       0.61130697, 0.4569382 , 0.51847094, 0.5913354 , 0.909069  ,
       0.9400851 , 0.22177139, 0.92203   , 0.62475055, 0.8744145 ,
       0.8844379 , 0.6734675 , 0.7462614 , 0.18068717, 0.34972283,
       0.58447665, 0.04109824, 0.3980274 , 0.52793413, 0.5740573 ],
      dtype=float32)